# Visualize and Analyze Startup Data

## 1. Setup

To prepare your environment, you need to install some packages and enter credentials for the Watson services.

## 1.1 Install the necessary packages

### Install Watson Developer Cloud Package

In [2]:
!pip install watson-developer-cloud==1.5

     |████████████████████████████████| 225kB 10.7MB/s eta 0:00:01
     |████████████████████████████████| 778kB 17.7MB/s eta 0:00:01
     |████████████████████████████████| 3.1MB 35.0MB/s eta 0:00:01
     |████████████████████████████████| 2.3MB 45.5MB/s eta 0:00:01
     |████████████████████████████████| 174kB 47.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 28.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 31.2MB/s eta 0:00:01
     |████████████████████████████████| 102kB 34.7MB/s ta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/81/cd/de/4e0916f623c2d125502e493394fd333ed693960264d4b7e524
Successfully built watson-developer-cloud
  Found existing installation: cryptography 2.5
    Uninstalling cryptography-2.5:
      Successfully uninstalled cryptography-2.5


### Install IBM Database Server Python Package

In [3]:
!pip install ibm_db

### Install Other Packages

In [4]:
!pip install cssselect

In [6]:
!pip install fake-useragent

## 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [7]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
from IPython.display import display, HTML
#import selenium
#from selenium import webdriver
import re
import pandas as pd

from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions, SemanticRolesOptions, SentimentOptions, EmotionOptions, ConceptsOptions, CategoriesOptions

import ibm_boto3
from botocore.client import Config


import json
import nltk
import csv
import ibm_db
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
from io import BytesIO

from urllib.parse import urlencode, urlparse, parse_qs

from lxml.html import fromstring
from requests import get
from fake_useragent import UserAgent

## 2. Configuration

Add configurable items of the notebook below

## 2.1 Add your service credentials from IBM Natural Language Understanding service
You must create a Watson Natural Language Understanding service on IBM Cloud. Create a service for Natural Language Understanding (NLU). Insert the apikey and url values in the variables, for your NLU in the following cell. Do not change the values of the version fields.

Run the cell.

In [8]:
apikey=''
url=''
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    iam_api_key=apikey,
    url=url
)

## 2.2 Add your service credentials for DB2

Insert the DB2 service credentials as credentials_1 in the following cell.

In [9]:

# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
}
    



## 3. Scrape Startup Information

Scrapes data based on its appears on google for the following -

* How many times it has appeared on News?
* Whether it has a Wikipedia page or not?
* Whether they have Tech blogs or not?
* Whether they are active on Social Media (Twitter, Medium, etc..)?


### Insert Pandas Dataframe of the `companies_list.json file`

Ensure the dataframe is saved as `df_data_1`

In [10]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_600a2a906fae469da3e34c688fbcf9e1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='FkFxDrMPJYQ72fR4HtD4DQeBFU_cA_ID5EuknlczIBMV',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_600a2a906fae469da3e34c688fbcf9e1.get_object(Bucket='scrapetrial-donotdelete-pr-fmuh1fzyvgwp8o',Key='companies_list_sample.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

df_data_1 = pd.read_json(body, orient='values')
df_data_1.head()



,companies
0,"{'IBM': ['International Business Machines', 'h..."
1,"{'Microsoft': ['Microsoft Corp', 'https://www...."


In [11]:
company_list_final=list(df_data_1['companies'])
company_list_final

[{'IBM': ['International Business Machines',
   'https://www.ibm.com/us-en/?ar=1']},
 {'Microsoft': ['Microsoft Corp', 'https://www.microsoft.com/en-us/']}]

In [12]:

def scrape_news_summaries_google(s):
    ua = UserAgent()
    
    number_result=10
    google_url = "https://www.google.com/search?q=" + s + "&num=" + str(number_result)
    response = requests.get(google_url, {"User-Agent": ua.random})
    soup = BeautifulSoup(response.text, "html.parser")

    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

    
    news_items=[]
    for r in result_div:
        # Checks if each element is present, else, raise exception
        
        try:
            news_dict=dict()
            link = r.find('a', href = True)
            title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
            description = r.find('div', attrs={'class':'s3v9rd'}).get_text()

            # Check to make sure everything is present before appending
            if link != '' and title != '' and description != '': 
                
                news_dict['news_link']=link['href']
                news_dict['summary']=description
                news_items.append(news_dict)
        # Next loop if one element is not present
        except:
            continue
    return news_items

In [13]:
final_rows=[]
val=dict()
for c in company_list_final:
    for key,value in c.items():
        s='"'+key+'"'+'company economic times'
        inner_dict=dict()
        temp=[]
        temp=temp+scrape_news_summaries_google(s)
        inner_dict['Description']=value[0]
        inner_dict['Company_Link']=value[1]
        inner_dict['News_Items']=temp
        val[key]=inner_dict
        final_rows.append(val)
        val=dict()

In [14]:
final_rows[0]

{'IBM': {'Description': 'International Business Machines',
  'Company_Link': 'https://www.ibm.com/us-en/?ar=1',
  'News_Items': [{'news_link': '/url?q=https://economictimes.indiatimes.com/topic/IBM&sa=U&ved=2ahUKEwja74L_lqXkAhVOd6wKHVtzCc4QFjAAegQIAxAB&usg=AOvVaw2h2obwbc-hDc9Cso7domxP',
    'summary': 'IBM: Find Latest Stories, Special Reports, News & Pictures on IBM. ... The technology company is facing several lawsuits accusing it of firing older workers.'},
   {'news_link': '/url?q=https://economictimes.indiatimes.com/tech/ites/reinventing-itself-says-ibm-on-reports-of-sacking-300-workers/articleshow/69325235.cms&sa=U&ved=2ahUKEwja74L_lqXkAhVOd6wKHVtzCc4QFjABegQICBAB&usg=AOvVaw3062kzg45pHQp_fP0XWWDe',
    'summary': 'May 14, 2019 · "IBM\'s strategy is to re-invent itself to better meet the changing requirements of our ... IBM had said in a statement that "We are proud of our company and our ... 600 senior employees in recent times, saying they could not adjust to the ... Download Th

In [15]:
def split_sentences(text):
    """ Split text into sentences.
    """
    sentence_delimiters = re.compile(u'[\\[\\]\n.!?]')
    sentences = sentence_delimiters.split(text)
    return sentences

def split_into_tokens(text):
    """ Split text into tokens.
    """
    tokens = nltk.word_tokenize(text)
    return tokens

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text.decode('utf-8')

def POS_tagging(text):
    """ Generate Part of speech tagging of the text.
    """
    POSofText = nltk.tag.pos_tag(text)
    return POSofText

def resolve_coreference(text, config):
    """ Resolve coreferences in the text for Nouns that are Subjects in a sentence
    """
    sentenceList = split_sentences(text)
    referenceSubject = ''
    sentenceText = ''
    configjson = json.loads(config)
    
    for sentences in sentenceList:    
        tokens = split_into_tokens(sentences)   
        postags = POS_tagging(tokens)
        sentencetags = chunk_sentence(postags)
        subjects = find_subject(sentencetags)
        for rules in configjson['configuration']['coreference']['rules']:
            if (rules['type'] == 'chunking'):
                for tags in rules['chunk']:
                    chunktags = chunk_tagging(tags['tag'],tags['pattern'],postags)
                    if (len(chunktags)>0):
                        for words in chunktags:
                            if tags['tag'] == 'PRP':
                                if subjects == '':
                                    sentenceText = sentenceText+sentences.replace(words,referenceSubject)+'. '
                            elif tags['tag'] == 'NAME':
                                if words == subjects:
                                    referenceSubject = words
                                    sentenceText = sentenceText+sentences+'. '
                    
    return sentenceText

def chunk_sentence(text):
    """ Tag the sentence using chunking.
    """
    grammar = """
      NP: {<DT|JJ|PRP|NN.*>+} # Chunk sequences of DT,JJ,NN
          #}<VB*|DT|JJ|RB|PRP><NN.*>+{  # Chink sequences of VB,DT,JJ,NN       
      PP: {<IN><NP>}               # Chunk prepositions followed by NP
      V: {<V.*>}                   # Verb      
      VP: {<VB*><NP|PP|CLAUSE>+}  # Chunk verbs and their arguments
      CLAUSE: {<NP><VP>}           # Chunk NP, VP
      """  
    parsed_cp = nltk.RegexpParser(grammar,loop=2)
    pos_cp = parsed_cp.parse(text)
    return pos_cp

def find_subject(t):
    for s in t.subtrees(lambda t: t.label() == 'NP'):
        return find_attrs(s,'NP')
    
def find_attrs(subtree,phrase):
    attrs = ''
    if phrase == 'NP':
        for nodes in subtree:
            if nodes[1] in ['DT','PRP$','POS','JJ','CD','ADJP','QP','NP','NNP']:
                attrs = attrs+' '+nodes[0]
    return attrs   

def chunk_tagging(tag,chunk,text):
    """ Tag the text using chunking.
    """
    parsed_cp = nltk.RegexpParser(chunk)
    pos_cp = parsed_cp.parse(text)
    chunk_list=[]
    for root in pos_cp:
        if isinstance(root, nltk.tree.Tree):               
            if root.label() == tag:
                chunk_word = ''
                for child_root in root:
                    chunk_word = chunk_word +' '+ child_root[0]
                chunk_list.append(chunk_word)
    return chunk_list

def analyze_using_NLU(analysistext):
    """ Extract results from Watson Natural Language Understanding for each news item
    """
    res=dict()
    response = natural_language_understanding.analyze( 
        text=analysistext,
        features=Features(
                          sentiment=SentimentOptions(),
                          entities=EntitiesOptions(), 
                          keywords=KeywordsOptions(),
                          emotion=EmotionOptions(),
                          concepts=ConceptsOptions(),
                          categories=CategoriesOptions(),
                          ))
    res['results']=response
    return res



In [16]:
def hasET(company_name):
    cnbcVal=0
    cnbcLinks=[]
    ET_link=[]
    s='"'+company_name+'"'+' economic times'
    res= scrape_news_summaries_google(s)
    return res

def hasTwitter(company_name):
    cnbcVal=0
    cnbcLinks=[]
    ET_link=[]
    s='"'+company_name+'"'+' twitter'
    res= scrape_news_summaries_google(s)
    return res


def getTechAreaNews(article_text):
    concept=''
    relevance=''
    if len(article_text) > 15:
        NLUres=analyze_using_NLU(article_text)
        
        if len(NLUres['results']['concepts']) != 0:
            concept=NLUres['results']['concepts'][0]['text']
            relevance=NLUres['results']['concepts'][0]['relevance']
        if len(NLUres['results']['sentiment']) != 0: 
            sentiment=NLUres['results']['sentiment']['document']['label']
    return concept,relevance,sentiment

def getTechArea(article_text):
    concept=''
    relevance=''
    sentiment=''
    if len(article_text) > 15:
        NLUres=analyze_using_NLU(article_text)
        if len(NLUres['results']['concepts']) != 0:
            concept=NLUres['results']['concepts'][0]['text']
            relevance=NLUres['results']['concepts'][0]['relevance']
    return concept,relevance

def hasWiki(s):
    wikiVal=0
    wikiLinks=[]
    s=s.replace(' ','+')
    link='https://en.wikipedia.org/w/index.php?search='+s+'&title=Special%3ASearch&go=Go'
    r = requests.get(link)
    print(r.status_code)
    content = r.text
    return content
    

## 3.1 Collect Wiki

Collects info on how many Companies have an existing Wikipedia page

In [17]:
wikiList=[]
for f in final_rows:
    for name, info in f.items():
        wiki=dict()
        wiki['Company_Name']=name
        wiki['Wiki_Concept'],wiki['Wiki_Confidence']=getTechArea(hasWiki(name))
        wikiList.append(wiki)
wikiList

200
200


[{'Company_Name': 'IBM', 'Wiki_Concept': 'IBM', 'Wiki_Confidence': 0.965767},
 {'Company_Name': 'Microsoft',
  'Wiki_Concept': 'Operating system',
  'Wiki_Confidence': 0.961208}]

In [18]:
keys = wikiList[0].keys()
with open('Wiki.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(wikiList)

## 3.2 Collect ET

Collects info through google links on how many hits are articles are from ET for a particular company

In [19]:
ET=[]
for f in final_rows:
    for name, info in f.items():
        temp=dict()
        news=hasET(name)
        for n in news:
            flag=0
            if 'summary' in n:
                summary=n['summary']
                flag=1
            link=n['news_link']
            temp=dict()

            if 'economictimes' in link and flag:
                    temp['Company_Name']=name
                    temp['News_Link']=link
                    temp['News_Concept'],temp['News_Relevance'],temp['News_Sentiment']=getTechAreaNews(summary)
                    ET.append(temp)
ET

[{'Company_Name': 'IBM',
  'News_Link': '/url?q=https://economictimes.indiatimes.com/topic/IBM&sa=U&ved=2ahUKEwip2reLl6XkAhUom-AKHa62CPAQFjAAegQIAxAB&usg=AOvVaw3ZvA1w3ojKjSBijxi4AEnE',
  'News_Concept': 'Broadsheet',
  'News_Relevance': 0.909961,
  'News_Sentiment': 'positive'},
 {'Company_Name': 'IBM',
  'News_Link': '/url?q=https://economictimes.indiatimes.com/topic/IBM-India/news&sa=U&ved=2ahUKEwip2reLl6XkAhUom-AKHa62CPAQFjABegQIBxAB&usg=AOvVaw1vT1ppPKPAnU7KWQqjf3hN',
  'News_Concept': 'Indian Standard Time',
  'News_Relevance': 0.863063,
  'News_Sentiment': 'positive'},
 {'Company_Name': 'IBM',
  'News_Link': '/url?q=https://economictimes.indiatimes.com/tech/ites/reinventing-itself-says-ibm-on-reports-of-sacking-300-workers/articleshow/69325235.cms&sa=U&ved=2ahUKEwip2reLl6XkAhUom-AKHa62CPAQFjACegQICRAB&usg=AOvVaw3MaO0XYe8r-FC7Lb8kJ3-z',
  'News_Concept': 'India',
  'News_Relevance': 0.922344,
  'News_Sentiment': 'positive'},
 {'Company_Name': 'IBM',
  'News_Link': '/url?q=https://e

In [20]:
keys = ET[0].keys()
with open('ET_final.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(ET)

## 3.3 Collect Tech Area

Suggests the major tech area of a company

In [21]:
tech_area=[]
for f in final_rows:
    for name,info in f.items():
        temp=dict()
        temp["Company_Name"]=name
        print(info)
        temp["Technology"],temp["Technology_Relevance"]=getTechArea(info['Description'])
        tech_area.append(temp)

{'Description': 'International Business Machines', 'Company_Link': 'https://www.ibm.com/us-en/?ar=1', 'News_Items': [{'news_link': '/url?q=https://economictimes.indiatimes.com/topic/IBM&sa=U&ved=2ahUKEwja74L_lqXkAhVOd6wKHVtzCc4QFjAAegQIAxAB&usg=AOvVaw2h2obwbc-hDc9Cso7domxP', 'summary': 'IBM: Find Latest Stories, Special Reports, News & Pictures on IBM. ... The technology company is facing several lawsuits accusing it of firing older workers.'}, {'news_link': '/url?q=https://economictimes.indiatimes.com/tech/ites/reinventing-itself-says-ibm-on-reports-of-sacking-300-workers/articleshow/69325235.cms&sa=U&ved=2ahUKEwja74L_lqXkAhVOd6wKHVtzCc4QFjABegQICBAB&usg=AOvVaw3062kzg45pHQp_fP0XWWDe', 'summary': 'May 14, 2019 · "IBM\'s strategy is to re-invent itself to better meet the changing requirements of our ... IBM had said in a statement that "We are proud of our company and our ... 600 senior employees in recent times, saying they could not adjust to the ... Download The Economic Times News A

In [22]:
tech_area

[{'Company_Name': 'IBM',
  'Technology': 'Computer',
  'Technology_Relevance': 0.867498},
 {'Company_Name': 'Microsoft', 'Technology': '', 'Technology_Relevance': ''}]

In [23]:
keys = tech_area[0].keys()
with open('Tech_Area_Final.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(tech_area)

## 3.4 Collect Twitter

Collects info on how many Tweets appear on Google Search of a Company

In [24]:
Twitter=[]
for f in final_rows:
    for name, info in f.items():
        temp=dict()
        news=hasTwitter(name)
        for n in news:
            flag=0
            if 'summary' in n:
                summary=n['summary']
                flag=1
            #print(summary)
            link=n['news_link']
            temp=dict()
            #print('economictimes' in link)
            if 'twitter.com/'+name.lower() in link and flag:
                    temp['Company_Name']=name
                    temp['Twitter_news_link']=link
                    temp['Twitter_Topic'],temp['Twitter_Relevance'],temp['Twitter_Sentiment']=getTechAreaNews(summary)
                    Twitter.append(temp)
Twitter

[{'Company_Name': 'IBM',
  'Twitter_news_link': '/url?q=https://twitter.com/ibmnews%3Flang%3Den&sa=U&ved=2ahUKEwiGhu-Vl6XkAhVLKa0KHQZ7B0kQFjACegQIBxAB&usg=AOvVaw03mNRjFvFhpPAArVBbQNQB',
  'Twitter_Topic': 'Twitter',
  'Twitter_Relevance': 0.886784,
  'Twitter_Sentiment': 'neutral'},
 {'Company_Name': 'IBM',
  'Twitter_news_link': '/url?q=https://twitter.com/ibmresearch%3Flang%3Den&sa=U&ved=2ahUKEwiGhu-Vl6XkAhVLKa0KHQZ7B0kQFjADegQIBhAB&usg=AOvVaw3Usa86n67fzrI4wVPHmbS0',
  'Twitter_Topic': 'Computer',
  'Twitter_Relevance': 0.904875,
  'Twitter_Sentiment': 'neutral'},
 {'Company_Name': 'IBM',
  'Twitter_news_link': '/url?q=https://twitter.com/ibmanalytics%3Flang%3Den&sa=U&ved=2ahUKEwiGhu-Vl6XkAhVLKa0KHQZ7B0kQFjAEegQICRAB&usg=AOvVaw2uVQWkRNoXUuFVwRW59zkG',
  'Twitter_Topic': '2009 albums',
  'Twitter_Relevance': 0.855269,
  'Twitter_Sentiment': 'positive'},
 {'Company_Name': 'IBM',
  'Twitter_news_link': '/url?q=https://twitter.com/ibmcloud%3Flang%3Den&sa=U&ved=2ahUKEwiGhu-Vl6XkAhVLKa0KHQ

In [25]:
keys = Twitter[0].keys()
with open('Twitter.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(Twitter)

## 4. Combine the Results and Save to SPSS

In [26]:
file = open("Tech_Area_Final.csv", "r")
Tech_area = pd.read_csv(file, delimiter=',')
file = open("ET_final.csv", "r")
ET = pd.read_csv(file, delimiter=',')
file = open("Wiki.csv", "r")
Wiki = pd.read_csv(file, delimiter=',')
file = open("Twitter.csv", "r")
Twitter = pd.read_csv(file, delimiter=',')

In [27]:
a = ET.append(Wiki)

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [28]:
b=a.append(Twitter)

In [29]:
b

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,NaN
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,NaN,NaN,NaN,NaN,NaN,NaN
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,NaN,NaN,NaN,NaN,NaN,NaN
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,NaN,NaN,NaN,NaN,NaN,NaN
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,NaN
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,NaN,NaN,NaN,NaN,NaN,NaN
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,NaN,NaN,NaN,NaN,NaN,NaN
7,Microsoft,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,neutral,NaN,NaN,NaN,NaN,NaN,NaN
8,Microsoft,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,negative,NaN,NaN,NaN,NaN,NaN,NaN
9,Microsoft,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,neutral,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
compiled_rows=pd.merge(b,Tech_area, on="Company_Name")

In [31]:
compiled_rows

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
7,IBM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBM,0.965767,Computer,0.867498
8,IBM,NaN,NaN,NaN,NaN,0.886784,neutral,Twitter,/url?q=https://twitter.com/ibmnews%3Flang%3Den...,NaN,NaN,Computer,0.867498
9,IBM,NaN,NaN,NaN,NaN,0.904875,neutral,Computer,/url?q=https://twitter.com/ibmresearch%3Flang%...,NaN,NaN,Computer,0.867498


In [32]:
compiled_rows

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,NaN,NaN,NaN,NaN,NaN,NaN,Computer,0.867498
7,IBM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBM,0.965767,Computer,0.867498
8,IBM,NaN,NaN,NaN,NaN,0.886784,neutral,Twitter,/url?q=https://twitter.com/ibmnews%3Flang%3Den...,NaN,NaN,Computer,0.867498
9,IBM,NaN,NaN,NaN,NaN,0.904875,neutral,Computer,/url?q=https://twitter.com/ibmresearch%3Flang%...,NaN,NaN,Computer,0.867498


In [33]:
compiled_rows['Wiki_Confidence'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
7,IBM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBM,0.965767,Computer,0.867498
8,IBM,NaN,NaN,NaN,NaN,0.886784,neutral,Twitter,/url?q=https://twitter.com/ibmnews%3Flang%3Den...,NaN,0.000000,Computer,0.867498
9,IBM,NaN,NaN,NaN,NaN,0.904875,neutral,Computer,/url?q=https://twitter.com/ibmresearch%3Flang%...,NaN,0.000000,Computer,0.867498


In [34]:
compiled_rows['News_Relevance'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
7,IBM,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,IBM,0.965767,Computer,0.867498
8,IBM,NaN,NaN,0.000000,NaN,0.886784,neutral,Twitter,/url?q=https://twitter.com/ibmnews%3Flang%3Den...,NaN,0.000000,Computer,0.867498
9,IBM,NaN,NaN,0.000000,NaN,0.904875,neutral,Computer,/url?q=https://twitter.com/ibmresearch%3Flang%...,NaN,0.000000,Computer,0.867498


In [35]:
compiled_rows['Technology_Relevance'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,NaN,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
7,IBM,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,IBM,0.965767,Computer,0.867498
8,IBM,NaN,NaN,0.000000,NaN,0.886784,neutral,Twitter,/url?q=https://twitter.com/ibmnews%3Flang%3Den...,NaN,0.000000,Computer,0.867498
9,IBM,NaN,NaN,0.000000,NaN,0.904875,neutral,Computer,/url?q=https://twitter.com/ibmresearch%3Flang%...,NaN,0.000000,Computer,0.867498


In [36]:
compiled_rows['Twitter_Relevance'].fillna(0.0, inplace=True)
compiled_rows

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
1,IBM,Indian Standard Time,/url?q=https://economictimes.indiatimes.com/to...,0.863063,positive,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
2,IBM,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,positive,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
3,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
4,IBM,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,positive,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
5,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
6,IBM,Information technology in India,/url?q=https://cio.economictimes.indiatimes.co...,0.949819,positive,0.000000,NaN,NaN,NaN,NaN,0.000000,Computer,0.867498
7,IBM,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,NaN,IBM,0.965767,Computer,0.867498
8,IBM,NaN,NaN,0.000000,NaN,0.886784,neutral,Twitter,/url?q=https://twitter.com/ibmnews%3Flang%3Den...,NaN,0.000000,Computer,0.867498
9,IBM,NaN,NaN,0.000000,NaN,0.904875,neutral,Computer,/url?q=https://twitter.com/ibmresearch%3Flang%...,NaN,0.000000,Computer,0.867498


In [37]:
import numpy as np
compiled_rows = compiled_rows.replace(np.nan, '', regex=True)

In [46]:
## Make sure compiled_rows.Company_Name.unique() <= 10. Since SPSS Modeller evaluates only 10 rows at a time
sample_len=int(10/len(list(compiled_rows.Company_Name.unique())))
sample_len

5

In [47]:
compiled_rows.groupby('Company_Name').apply(lambda x: x.sample(sample_len)).reset_index(drop=True)

,Company_Name,News_Concept,News_Link,News_Relevance,News_Sentiment,Twitter_Relevance,Twitter_Sentiment,Twitter_Topic,Twitter_news_link,Wiki_Concept,Wiki_Confidence,Technology,Technology_Relevance
0,IBM,August,/url?q=https://economictimes.indiatimes.com/ne...,0.844918,negative,0.000000,,,,,0.000000,Computer,0.867498
1,IBM,,,0.000000,,0.855269,positive,2009 albums,/url?q=https://twitter.com/ibmanalytics%3Flang...,,0.000000,Computer,0.867498
2,IBM,Hebrew numerals,/url?q=https://economictimes.indiatimes.com/te...,0.863236,neutral,0.000000,,,,,0.000000,Computer,0.867498
3,IBM,,,0.000000,,0.000000,positive,,/url?q=https://twitter.com/ibmdesign%3Flang%3D...,,0.000000,Computer,0.867498
4,IBM,,,0.000000,,0.000000,,,,IBM,0.965767,Computer,0.867498
5,Microsoft,World,/url?q=https://economictimes.indiatimes.com/ne...,0.911360,negative,0.000000,,,,,0.000000,,0.000000
6,Microsoft,Broadsheet,/url?q=https://economictimes.indiatimes.com/to...,0.909961,neutral,0.000000,,,,,0.000000,,0.000000
7,Microsoft,,,0.000000,,0.945319,neutral,Customer service,/url?q=https://twitter.com/microsofthelps%3Fla...,,0.000000,,0.000000
8,Microsoft,Computer data storage,/url?q=https://economictimes.indiatimes.com/te...,0.930897,neutral,0.000000,,,,,0.000000,,0.000000
9,Microsoft,India,/url?q=https://economictimes.indiatimes.com/te...,0.922344,negative,0.000000,,,,,0.000000,,0.000000


### Store and Add table in DB2 Warehouse 

In [142]:
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = credentials_1['db'] 
dsn_hostname = credentials_1['host']
dsn_port = 50000               
dsn_uid = credentials_1['username']      
dsn_pwd = credentials_1['password']

dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE="+str(dsn_database)+";"
    "HOSTNAME="+str(dsn_hostname)+";"
    "PORT="+str(dsn_port)+";"
    "PROTOCOL=TCPIP;"
    "UID="+str(dsn_uid)+";"
    "PWD="+str(dsn_pwd)+";").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")

In [143]:
create_statement=pd.io.sql.get_schema(compiled_rows.reset_index(), 'DATA_FOR_SPSS')
create_statement=create_statement.replace('TEXT', 'VARCHAR(500)')
ibm_db.exec_immediate(conn, create_statement)

In [145]:
tuple_of_tuples = tuple([tuple(x) for x in compiled_rows.values])
i=1
for x in compiled_rows.values:
    vals= (i,) + tuple(x)
    print(vals)
    sql = "INSERT INTO "+dsn_uid+".DATA_FOR_SPSS VALUES"+ str(vals)
    i=i+1
    ins_sql=ibm_db.prepare(conn, sql)
    ibm_db.execute(ins_sql)


(1, 'IBM', 'Broadsheet', '/url?q=https://economictimes.indiatimes.com/topic/IBM&sa=U&ved=2ahUKEwiHlc7a66TkAhVnhq0KHV9TBuoQFjAAegQIARAB&usg=AOvVaw2ufU5dG51DbBkkXOhF3mPL', 0.909961, 'positive', 0.0, '', '', '', '', 0.0, 'Computer', 0.867498)
(2, 'IBM', 'India', '/url?q=https://economictimes.indiatimes.com/tech/ites/reinventing-itself-says-ibm-on-reports-of-sacking-300-workers/articleshow/69325235.cms&sa=U&ved=2ahUKEwiHlc7a66TkAhVnhq0KHV9TBuoQFjABegQIChAB&usg=AOvVaw3dJv4aFhcGlNZtIVY0vk1z', 0.922344, 'positive', 0.0, '', '', '', '', 0.0, 'Computer', 0.867498)
(3, 'IBM', 'Broadsheet', '/url?q=https://economictimes.indiatimes.com/topic/IBM-India&sa=U&ved=2ahUKEwiHlc7a66TkAhVnhq0KHV9TBuoQFjACegQICRAB&usg=AOvVaw0gFwIf3jd9hLTjiTuZJ6yb', 0.909961, 'positive', 0.0, '', '', '', '', 0.0, 'Computer', 0.867498)
(4, 'IBM', 'Broadsheet', '/url?q=https://economictimes.indiatimes.com/topic/IBM-Watson&sa=U&ved=2ahUKEwiHlc7a66TkAhVnhq0KHV9TBuoQFjADegQICBAB&usg=AOvVaw3O3m3c5LUoqBTOQNEhwzPw', 0.909961, 'posi

In [146]:
create_statement= 'CREATE TABLE "DATA_FOR_COGNOS" (\n"index" SMALLINT,\n "Company_Name" VARCHAR,\n  "News_Concept" VARCHAR,\n  "News_Link" VARCHAR, \n  "News_Relevance" DECFLOAT,\n "Overall_Sentiment" VARCHAR ,\n  "Twitter_Topic" VARCHAR,\n  "Twitter_news_link" VARCHAR,\n  "Wiki_Concept" VARCHAR,\n "Wiki_Confidence" VARCHAR,\n  "Technology" VARCHAR,\n  "Technology_Relevance" DECFLOAT,\n "Company_News_Sentiments" SMALLINT\n)'
create_statement=create_statement.replace('VARCHAR', 'VARCHAR(500)')
ibm_db.exec_immediate(conn, create_statement)